# Homework_2

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse

from pathlib import Path
BASE_PATH = str(Path.cwd().parent) # BASE_PATH: /Users/stvbo/git/link_dl
print("BASE_PATH:", BASE_PATH)

import sys
sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset \
    import get_preprocessed_dataset

C:\Users\stvbo\anaconda3\envs\link_dl\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
C:\Users\stvbo\anaconda3\envs\link_dl\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

BASE_PATH: C:\Users\stvbo\git\link_dl


# 요구사항 1
- Titanic 데이터 로드

In [3]:
def get_data():
  train_dataset, validation_dataset, _ = get_preprocessed_dataset()
  print(len(train_dataset), len(validation_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

  return train_data_loader, validation_data_loader

In [4]:
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ReLU(), # ReLU & Batch size (16) 일 때 가장 결과가 좋음.
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

- `MyModel(n_input=10, n_output=1)`
- titanic 데이터셋에 맞게 feature 수 변경

In [5]:
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=1)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

In [15]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.MSELoss()  # Use a built-in loss function
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target']
      output_train = model(input)
      target = target.unsqueeze(1).float()
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target']
        output_validation = model(input)
        target = target.unsqueeze(1).float()
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

In [16]:
class Args:
    wandb = True  # wandb 사용 여부
    batch_size = 16
    epochs = 1000

args=Args()

current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

config = {
'epochs': args.epochs,
'batch_size': args.batch_size,
'learning_rate': 1e-3,
'n_hidden_unit_list': [20, 20],
}

wandb.init(
mode="online" if args.wandb else "disabled",
project="my_model_training",
notes="My first wandb experiment",
tags=["my_model", "california_housing"],
name=current_time_str,
config=config
)
print(args)
print(wandb.config)

train_data_loader, validation_data_loader = get_data()

linear_model, optimizer = get_model_and_optimizer()

print("#" * 50, 1)

training_loop(
model=linear_model,
optimizer=optimizer,
train_data_loader=train_data_loader,
validation_data_loader=validation_data_loader
)
wandb.finish()

# https://docs.wandb.ai/guides/track/config

{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}


C:\Users\stvbo\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\stvbo\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
Training loss,██▇▇▇▆▆▆▆▄▄▄▄▄▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁
Validation loss,▇▇▇█▆▅▅▅▅▄▃▆▃▃▂▃▃▂▂▂▄▂▃▂▂▁▁▂▁▁▂▇▂▁▁▂▂▂▁▁
Epoch,1000
Training loss,0.14177
Validation loss,0.13029


# 요구사항 2

- 더 나은 성능을 산출하는 Activation function 조사 (Batch Size 16 기준)
    - 결과 : ReLU를 사용했을 때, 가장 좋은 성능을 산출. (Training loss:0.14047410405344432
Validation loss:0.1235535964369774)
    - ![Sigmoid](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/Sigmoid_16.png "Sigmoid_16") (Sigmoid)
    - ![ReLU](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/ReLU_16.png "ReLU_16") (ReLU)
    - ![ELU](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/ELU_16.png "ReLU_16") (ELU)
    - ![Lealy ReLU](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/Lealy_ReLU_16.png "Lealy ReLU") (Lealy ReLU)
- 더 나은 성능을 산출하는 Batch Size 조사 (Activation function : ReLU 기준)
    - 결과 : Batch Size 16일 때, 가장 좋은 성능을 산출. (Training loss:0.13184196170833376
Validation loss:0.13638935983181)
    - ![Size 16](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/16_ReLU.png "Size 16") (Size 16)
    - ![Size 32](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/32_ReLU.png "Size 32") (Size 32)
    - ![Size 64](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/64_ReLU.png "Size 64") (Size 64)
    - ![Size 128](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/128_ReLU.png "Size 128") (Size 128)

# 요구사항 3
- Activation Function : ReLU
- Batch Size : 16

In [6]:
from _01_code._99_common_utils.early_stopping import EarlyStopping

- training_loop : EarlyStopping 객체를 사용해, 90 epoch 기간을 부여하여 오버피팅 되기 전의 모델 추출
- 문제 : 모델의 Loss값의 변동 추이가 너무 불규칙하여 오버피팅되는 시점을 잡기 어려움. (아직 해결 못함)

In [33]:
import os

def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
  checkpoint_file_path = os.path.join('checkpoint')
  early_stopping = EarlyStopping(90, 0.00001, 'my_model', checkpoint_file_path)
  n_epochs = wandb.config.epochs
  loss_fn = nn.MSELoss()  # Use a built-in loss function
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target']
      output_train = model(input)
      target = target.unsqueeze(1).float()
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target']
        output_validation = model(input)
        target = target.unsqueeze(1).float()
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1
        message, early_stop = early_stopping.check_and_save(loss_validation / num_validations, model)

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

    if early_stop:
          break


- test dataset을 받아오도록 get_data() 함수 재정의

In [20]:
def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(len(train_dataset), len(validation_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader

- 테스트를 수행하고 Kaggle 제출 형식의 CSV 파일로 저장하는 함수.

In [35]:
import pandas as pd
import torch

def generate_submission_file(model, test_data_loader, checkpoint_file_path):
    # 1. 원본 테스트 데이터의 PassengerId를 가져오기 위해 'test.csv' 파일을 로드.
    try:
        base_path = os.path.dirname(os.getcwd())
        test_csv_path = os.path.join(base_path, '_03_homeworks', 'homework_2', 'test.csv')
        print(f"Attempting to load test.csv from: {test_csv_path}")
        test_df_original = pd.read_csv(test_csv_path)
    except Exception as e:
        print(f"Error loading 'test.csv': {e}")
        print("Assuming 'test.csv' is in the current working directory.")
        test_df_original = pd.read_csv("test.csv")

    # 2. 모델 상태 불러오기
    model_path = os.path.join(checkpoint_file_path, "my_model_checkpoint_latest.pt")
    print(f"Loading model from: {model_path}")
    model.load_state_dict(torch.load(model_path))
    model.eval()

    all_predictions = []
    # 4. 테스트 데이터로더를 순회하며 예측을 수행합니다.
    with torch.no_grad():  # 그래디언트 계산을 비활성화하여 메모리 사용량과 계산 속도를 최적화합니다.
        for batch in test_data_loader:
            inputs = batch['input']
            outputs = model(inputs)

            # 모델 출력이 0~1 사이의 값이므로, 0.5를 기준으로 생존(1)과 사망(0)을 결정합니다.
            predicted = (outputs > 0.5).long()

            # 예측 결과를 리스트에 추가합니다.
            all_predictions.extend(predicted.squeeze().cpu().tolist())

    # 5. 제출 형식에 맞게 DataFrame을 생성합니다.
    submission_df = pd.DataFrame({
        "PassengerId": test_df_original["PassengerId"],
        "Survived": all_predictions
    })

    # 6. DataFrame을 CSV 파일로 저장합니다. index=False로 불필요한 인덱스 열을 제거합니다.
    submission_path = "submission.csv"
    submission_df.to_csv(submission_path, index=False)

    print("-" * 50)
    print(f"Submission file has been successfully created at: {submission_path}")
    print("Submission file head:")
    print(submission_df.head())

In [37]:
class Args:
    wandb = True  # wandb 사용 여부
    batch_size = 16
    epochs = 1000

args=Args()

current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

config = {
'epochs': args.epochs,
'batch_size': args.batch_size,
'learning_rate': 1e-3,
'n_hidden_unit_list': [20, 20],
}

wandb.init(
mode="online" if args.wandb else "disabled",
project="my_model_training",
notes="My first wandb experiment",
tags=["my_model", "california_housing"],
name=current_time_str,
config=config
)
print(args)
print(wandb.config)

train_data_loader, validation_data_loader, test_data_loader = get_data()

linear_model, optimizer = get_model_and_optimizer()

print("#" * 50, 1)

training_loop(
model=linear_model,
optimizer=optimizer,
train_data_loader=train_data_loader,
validation_data_loader=validation_data_loader,
test_data_loader=test_data_loader
)

checkpoint_folder = 'checkpoint'
generate_submission_file(linear_model, test_data_loader, checkpoint_folder)

wandb.finish()


{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}


C:\Users\stvbo\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\stvbo\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
Training loss,█▆▆▅▅▅▅▅▅▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▁▂▁▁▂▁▂▁▁▂▁▁▁▂▁
Validation loss,▇▇▇▆▅▅▄▄▄▄▃▃▃▃▃▄▅▂▄▅█▃▂▂▂▃▁▁▃▁▂▂▃▁▂▂▃▁▂▂
Epoch,1000
Training loss,0.13797
Validation loss,0.1383


# 요구사항 4

![Kaggle Grade](https://github.com/Dong-kyu-Lee/Deep-Learning-Class-Practice/raw/master/img/Kaggle_grade.png "Kaggle Grade")

등수 : 13179
이름 : Dong Gyu Lee
Score : 0.74401
Entries : 1
Last : 2m